# Rescue M. smithii growth in minimal medium #

In [1]:
import pycomo
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import warnings
import os
import cobra
from itertools import combinations 

sb.set_theme()

2025-11-17 13:53:56,956 - pycomo - INFO - Logger initialized.
2025-11-17 13:53:56,960 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 13:53:56,961 - pycomo - INFO - Utils Logger initialized.
2025-11-17 13:53:56,964 - pycomo - INFO - Multiprocess Logger initialized.


## Load model ##

In [2]:
model_path = "data/community_models/M. smithii_B. thetaiotaomicron_ms_open_min_med_tp_10.xml"
com_model = pycomo.CommunityModel.load(model_path)

## Load media ##

In [3]:
min_medium_community = pycomo.read_medium_from_file("data/media/minimal_medium_community.csv", "_medium")


In [4]:
ms_medium = pycomo.read_medium_from_file("data/media/ms_b_tim_medium_gapseq_open.csv", "_medium")
for k, v in ms_medium.items():
    ms_medium[k] = 1000.

## Identify M. smithii minimal medium (single culture) ##

In [5]:
member_names = com_model.get_member_names()
com_model.convert_to_fixed_abundance()
com_model.apply_fixed_abundance({"NC_004663_1_ms_b_tim_medium_gapseq_open":0., "ABYW00000000_1_ms_medium_gapseq_open":1.})
com_model.medium = ms_medium
com_model.apply_medium()
com_model.model.reactions.abundance_reaction.bounds = 0.,1000.

2025-11-17 13:54:11,122 - pycomo - INFO - Note: Model already has fixed abundance structure.


In [6]:
m_smithii_min_medium = cobra.medium.minimal_medium(com_model.model)

medium_dict = m_smithii_min_medium.to_dict()

medium_df_rows = []

for exchg_rxn in medium_dict.keys():
    if exchg_rxn == "abundance_reaction": continue
    met = list(com_model.model.reactions.get_by_id(exchg_rxn).metabolites.keys())[0]
    met_id = met.id.replace("_medium","")
    met_name = met.name.replace("_medium", "")
    medium_df_rows.append({"compounds": met_id, "name": met_name, "maxFlux": 1000})

medium_df = pd.DataFrame.from_dict(medium_df_rows)

medium_df.to_csv("data/media/minimal_medium_m_smithii.csv", index=False, sep=",")

medium_df

,compounds,name,maxFlux
0,cpd00009,Phosphate,1000
1,cpd00051,L-Arginine,1000
2,cpd00098,Choline,1000
3,cpd00119,L-Histidine,1000
4,cpd00129,L-Proline,1000
5,cpd00149,Co2+,1000
6,cpd00156,L-Valine,1000
7,cpd00161,L-Threonine,1000
8,cpd00244,Ni2+,1000
9,cpd00092,Uracil,1000


### Find essential metabolites of M. smithii ###

For this analysis, all potential substrates of M. smithii are allowed, without restriction to the MS medium

In [7]:
member_names

['ABYW00000000_1_ms_medium_gapseq_open',
 'NC_004663_1_ms_b_tim_medium_gapseq_open']

In [8]:
all_mets_medium = {}
for rxn in com_model.model.reactions.query(lambda x: member_names[0] + "_TF_" in x.id):
    met_id = rxn.id.split(member_names[0] + "_TF_")[1].split("_")[0]
    
    all_mets_medium[f"EX_{met_id}_medium"] = 1000.

In [9]:
essentials = {}
for rxn in all_mets_medium.keys():
    test_med = all_mets_medium.copy()
    test_med[rxn] = 0.
    com_model.apply_fixed_abundance({"NC_004663_1_ms_b_tim_medium_gapseq_open":0., "ABYW00000000_1_ms_medium_gapseq_open":1.})
    com_model.medium = test_med
    com_model.apply_medium()
    com_model.model.reactions.abundance_reaction.bounds = 0.,1000.
    sol = com_model.model.optimize()
    if sol.objective_value < 0.001:
        print(f"Essential metabolite found! Growth after deletion of {rxn}: {sol.objective_value}")
        essentials[rxn] = sol.objective_value

Essential metabolite found! Growth after deletion of EX_cpd00129_medium: -4.0097326775236434e-15
Essential metabolite found! Growth after deletion of EX_cpd00149_medium: 0.0
Essential metabolite found! Growth after deletion of EX_cpd00161_medium: -5.645607412464698e-15
Essential metabolite found! Growth after deletion of EX_cpd00244_medium: -1.4114018531161743e-14
Essential metabolite found! Growth after deletion of EX_cpd00393_medium: -3.3619592141227267e-12


In [10]:
medium_df_rows = []

for exchg_rxn in essentials.keys():
    if exchg_rxn == "abundance_reaction": continue
    met = list(com_model.model.reactions.get_by_id(exchg_rxn).metabolites.keys())[0]
    met_id = met.id.replace("_medium","")
    met_name = met.name.replace("_medium", "")
    medium_df_rows.append({"compounds": met_id, "name": met_name, "maxFlux": 1000})

medium_df = pd.DataFrame.from_dict(medium_df_rows)

medium_df.to_csv("data/media/essential_metabolites_m_smithii.csv", index=False, sep=",")

medium_df

,compounds,name,maxFlux
0,cpd00129,L-Proline,1000
1,cpd00149,Co2+,1000
2,cpd00161,L-Threonine,1000
3,cpd00244,Ni2+,1000
4,cpd00393,Folate,1000


In [11]:
essentials_not_in_min_com = []
for k in essentials.keys():
    if k not in min_medium_community:
        print(f"{k} not in community minimal medium!")
        essentials_not_in_min_com.append(k)

EX_cpd00129_medium not in community minimal medium!
EX_cpd00161_medium not in community minimal medium!


## Rescue M. smithii growth in community minimal medium ##

With a list of essential metabolites at hand, the minimal medium can be supplemented for growth of M. smithii. Since L-Proline and L-Threonine are not in the minimal medium, they must be added to the list of supplements.

In [12]:
base_supplements = ("EX_cpd00129_medium", "EX_cpd00161_medium")

In [13]:
min_med_community_supp = min_medium_community.copy()
for rxn in base_supplements:
    min_med_community_supp[rxn] = 1000.


Now we can check for growth with the supplements

In [14]:
test_med = min_med_community_supp

com_model.apply_fixed_abundance({"NC_004663_1_ms_b_tim_medium_gapseq_open":0., "ABYW00000000_1_ms_medium_gapseq_open":1.})
com_model.medium = test_med
com_model.apply_medium()
com_model.model.reactions.abundance_reaction.bounds = 0.,1000.
sol = com_model.model.optimize()
print(sol.objective_value)

1.3925395249745304e-17


Still no growth! Likely, some metabolites need to be added, but might not be deleterious when they are the only metabolite missing from the medium (i.e. no single-KO).

### Finding supplement combinations to rescue the growth ###

In [15]:
m_smithii_mets_not_in_min_medium = []
for k in all_mets_medium.keys():
    if k not in min_medium_community:
        m_smithii_mets_not_in_min_medium.append(k)

print(f"There are {len(m_smithii_mets_not_in_min_medium)} metabolites in the model missing from the minimal community medium.")

There are 121 metabolites in the model missing from the minimal community medium.


In [16]:
solutions = {}
test_mets = m_smithii_mets_not_in_min_medium.copy()
test_mets.remove("EX_cpd00129_medium")
test_mets.remove("EX_cpd00161_medium")

for num_components in range(1,3):
    print(f"Testing {num_components} components")
    for i, combination in enumerate(combinations(m_smithii_mets_not_in_min_medium, num_components)):
        combination = ("EX_cpd00129_medium", "EX_cpd00161_medium") + combination
        test_med = min_medium_community.copy()
        for k in combination:
            test_med[k] = 1000.
            
        com_model.apply_fixed_abundance({"NC_004663_1_ms_b_tim_medium_gapseq_open":0., "ABYW00000000_1_ms_medium_gapseq_open":1.})
        com_model.medium = test_med
        com_model.apply_medium()
        com_model.model.reactions.abundance_reaction.bounds = 0.,1000.
        sol = com_model.model.optimize()
        if sol.objective_value > 0.001:
            print(f"Rescued with growth of {sol.objective_value}")
            solutions[combination] = sol.objective_value
        if (i+1) % 100 == 0:
            print(f"Tested {i+1} combinations")

Testing 1 components
Tested 100 combinations
Testing 2 components
Rescued with growth of 0.22321434801986828
Rescued with growth of 0.22319845774180566
Tested 100 combinations
Tested 200 combinations
Tested 300 combinations
Tested 400 combinations
Tested 500 combinations
Tested 600 combinations
Tested 700 combinations
Tested 800 combinations
Tested 900 combinations
Tested 1000 combinations
Tested 1100 combinations
Tested 1200 combinations
Tested 1300 combinations
Tested 1400 combinations
Tested 1500 combinations
Tested 1600 combinations
Tested 1700 combinations
Tested 1800 combinations
Tested 1900 combinations
Tested 2000 combinations
Tested 2100 combinations
Tested 2200 combinations
Tested 2300 combinations
Tested 2400 combinations
Tested 2500 combinations
Tested 2600 combinations
Tested 2700 combinations
Tested 2800 combinations
Tested 2900 combinations
Tested 3000 combinations
Rescued with growth of 0.006343947353637299
Tested 3100 combinations
Rescued with growth of 0.0063436167436

In [20]:
print(f"There are {len(solutions)} distinct supplement combinations of size 4 that rescue growth of M. smithii")

There are 36 distinct supplement combinations of size 4 that rescue growth of M. smithii


In [22]:
for i, sol in enumerate(solutions.keys()):
    medium_df_rows = []
    
    for exchg_rxn in sol:
        if exchg_rxn == "abundance_reaction": continue
        met = list(com_model.model.reactions.get_by_id(exchg_rxn).metabolites.keys())[0]
        met_id = met.id.replace("_medium","")
        met_name = met.name.replace("_medium", "")
        medium_df_rows.append({"compounds": met_id, "name": met_name, "maxFlux": 1000})
    
    medium_df = pd.DataFrame.from_dict(medium_df_rows)
    
    medium_df.to_csv(f"data/media/rescue_supplements_{i+1}_m_smithii.csv", index=False, sep=",")